### MID 11
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### SVD used for feature reduction, n_components = 10.
### Oversampling and smote methods used to increase size of training data.
### RNN model used, model type is 4 as per the BTP documentation spreadsheet.
### np.rint() used on predictions

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
import csv
from sklearn.decomposition import TruncatedSVD

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [4]:
combined_data

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.3,org.apache.tools.ant.taskdefs.ExecuteOn,0.142857,0.6,0.050,0.102941,0.214286,0.011856,0.014815,...,1.000000,0.111111,0.885057,0.232323,0.75,0.363636,0.165951,0.085714,0.209085,0
1,ant,1.3,org.apache.tools.ant.DefaultLogger,0.181818,0.0,0.025,0.058824,0.163265,0.020033,0.029630,...,1.000000,0.000000,0.000000,0.307692,0.00,0.000000,0.080979,0.171429,0.269903,2
2,ant,1.3,org.apache.tools.ant.taskdefs.TaskOutputStream,0.038961,0.2,0.000,0.007353,0.045918,0.000000,0.000000,...,1.000000,0.111111,0.714286,0.666667,0.25,0.090909,0.083267,0.028571,0.109529,0
3,ant,1.3,org.apache.tools.ant.taskdefs.Cvs,0.155844,0.4,0.000,0.088235,0.188776,0.013083,0.000000,...,1.000000,0.111111,0.770833,0.458333,0.00,0.000000,0.115693,0.085714,0.232742,0
4,ant,1.3,org.apache.tools.ant.taskdefs.Copyfile,0.077922,0.4,0.000,0.029412,0.107143,0.000409,0.000000,...,1.000000,0.000000,0.880952,0.416667,0.50,0.181818,0.100881,0.028571,0.136898,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,0.064935,0.4,0.000,0.051471,0.096939,0.001635,0.029630,...,0.000000,0.000000,0.902439,0.400000,0.25,0.090909,0.138351,0.028571,0.131428,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,0.220779,0.0,0.075,0.110294,0.336735,0.036795,0.029630,...,1.000000,0.222222,0.000000,0.197917,0.00,0.000000,0.172373,0.400000,0.396221,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,0.298701,0.0,0.000,0.227941,0.418367,0.082175,0.162963,...,0.444444,0.000000,0.000000,0.318182,0.00,0.000000,0.219724,0.685714,0.378561,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,0.142857,0.0,0.200,0.161765,0.438776,0.006132,0.059259,...,1.000000,0.777778,0.000000,0.266667,0.00,0.000000,0.373390,0.628571,0.761689,1


In [5]:
# define transform
components = 10
svd = TruncatedSVD(n_components=components)
# prepare transform on dataset
svd.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = svd.transform(combined_data[cols_to_norm])

In [6]:
transformed = pd.DataFrame(transformed)
transformed 

,0,1,2,3,4,5,6,7,8,9
0,1.697270,-0.240716,0.571264,0.167873,-0.202270,-0.188764,-0.085932,0.036019,-0.075786,0.129869
1,1.016611,-0.267720,-0.587890,-0.054598,-0.014350,-0.077750,-0.160155,0.052980,0.100634,-0.030916
2,1.298451,-0.173527,0.139837,-0.541003,-0.007964,0.011583,0.156168,-0.110641,-0.028186,0.008754
3,1.537891,-0.056428,0.028502,-0.104266,0.280907,0.130204,-0.176106,-0.029262,-0.088801,-0.025663
4,1.530257,-0.074334,0.374525,-0.205480,-0.061914,-0.152907,-0.063597,0.021766,0.089365,0.037752
...,...,...,...,...,...,...,...,...,...,...
298,0.886501,0.524176,0.551332,0.103135,0.083555,0.172359,0.095620,0.012480,0.004141,0.017085
299,1.128822,-0.426056,-0.629426,0.258850,-0.034627,0.177394,-0.220035,0.025682,-0.128906,0.061409
300,1.027511,-0.126484,-0.591917,0.769665,0.027848,0.483424,0.049615,-0.134999,-0.142810,0.233455
301,1.318002,-0.514244,-0.775534,0.478137,-0.191742,0.531927,-0.176518,0.103397,-0.589727,0.040915


In [7]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [8]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [9]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)

In [10]:
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

In [11]:
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [12]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)

train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [13]:
train_x

array([[[ 1.69726979, -0.24071623,  0.57126395, ...,  0.03601945,
         -0.07578644,  0.1298695 ]],

       [[ 1.01661138, -0.26772002, -0.58788997, ...,  0.05298001,
          0.10063434, -0.03091627]],

       [[ 1.2984506 , -0.17352675,  0.13983707, ..., -0.1106405 ,
         -0.02818633,  0.00875374]],

       ...,

       [[ 1.01420045, -0.37421302, -0.54721556, ...,  0.0060236 ,
          0.0477774 ,  0.1532802 ]],

       [[ 1.01420045, -0.37421302, -0.54721556, ...,  0.0060236 ,
          0.0477774 ,  0.1532802 ]],

       [[ 1.53857934, -0.22516725,  0.44500853, ...,  0.01491268,
         -0.05165907,  0.11134909]]])

In [14]:
Y_train

0      0
1      2
2      0
3      0
4      0
      ..
415    3
416    3
417    3
418    3
419    3
Name: bug, Length: 420, dtype: int64

In [15]:
train_y = Y_train.to_numpy()
train_y

array([0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 3, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [16]:
test_x

array([[[ 1.52329275e+00, -1.70090313e-01,  1.28455989e-01, ...,
         -6.80751641e-04, -8.03932395e-02, -4.73045611e-02]],

       [[ 1.65167769e+00, -6.32547908e-02,  3.68564825e-01, ...,
         -8.72648436e-02,  3.00134262e-02,  2.13725871e-01]],

       [[ 1.02228015e+00, -1.47043172e-01, -5.94705710e-01, ...,
         -7.07956961e-02,  6.31230009e-02, -9.62148357e-03]],

       ...,

       [[ 1.02751126e+00, -1.26484498e-01, -5.91917396e-01, ...,
         -1.34998564e-01, -1.42810088e-01,  2.33454670e-01]],

       [[ 1.31800237e+00, -5.14243570e-01, -7.75534220e-01, ...,
          1.03397226e-01, -5.89726796e-01,  4.09146172e-02]],

       [[ 1.53449933e+00,  3.23618749e-02,  6.88108962e-02, ...,
         -8.55739502e-02,  1.46074654e-02, -3.37952553e-02]]])

In [17]:
Y_test

0      0
1      0
2      0
3      0
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 178, dtype: int64

In [18]:
test_y = Y_test.to_numpy()
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0])

In [19]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,components), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = 'adam' , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 100)            11100     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 1, 80)             14480     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 60)                8460      
_________________________________________________________________
dense (Dense)                (None, 1)                 61        
Total params: 34,101
Trainable params: 34,101
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Fitting the model on training data.
import time
t0=time.time()
history = model.fit(train_x, train_y, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 6ms/step - loss: 2.3682 - mse: 2.3682 - mae: 1.2515 - root_mean_squared_error: 1.5389 - mean_squared_logarithmic_error: 0.5109
Epoch 2/100
4/4 [==============================] - 0s 6ms/step - loss: 1.5290 - mse: 1.5290 - mae: 1.0143 - root_mean_squared_error: 1.2365 - mean_squared_logarithmic_error: 0.3441
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 1.7054 - mse: 1.7054 - mae: 1.0332 - root_mean_squared_error: 1.3059 - mean_squared_logarithmic_error: 0.3650
Epoch 4/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2928 - mse: 1.2928 - mae: 0.9563 - root_mean_squared_error: 1.1370 - mean_squared_logarithmic_error: 0.2858
Epoch 5/100
4/4 [==============================] - 0s 5ms/step - loss: 1.2788 - mse: 1.2788 - mae: 0.9543 - root_mean_squared_error: 1.1308 - mean_squared_logarithmic_error: 0.2564
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 1.1745 - mse: 1.1745 - m

4/4 [==============================] - 0s 5ms/step - loss: 0.7830 - mse: 0.7830 - mae: 0.7287 - root_mean_squared_error: 0.8849 - mean_squared_logarithmic_error: 0.1802
Epoch 47/100
4/4 [==============================] - 0s 6ms/step - loss: 0.8172 - mse: 0.8172 - mae: 0.7543 - root_mean_squared_error: 0.9040 - mean_squared_logarithmic_error: 0.1802
Epoch 48/100
4/4 [==============================] - 0s 6ms/step - loss: 0.8092 - mse: 0.8092 - mae: 0.7532 - root_mean_squared_error: 0.8995 - mean_squared_logarithmic_error: 0.1830
Epoch 49/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8308 - mse: 0.8308 - mae: 0.7419 - root_mean_squared_error: 0.9115 - mean_squared_logarithmic_error: 0.1987
Epoch 50/100
4/4 [==============================] - 0s 5ms/step - loss: 0.7896 - mse: 0.7896 - mae: 0.7364 - root_mean_squared_error: 0.8886 - mean_squared_logarithmic_error: 0.1824
Epoch 51/100
4/4 [==============================] - 0s 5ms/step - loss: 0.8279 - mse: 0.8279 - mae: 0.7

In [ ]:
print("training time:", round(time.time()-t0, 3), "s") # the time would be round to 3 decimal in seconds

In [22]:
# Getting the predictions.
t1=time.time()
predictions_y = model.predict(test_x)
print("predict time:", round(time.time()-t1, 3), "s")
predictions_y

array([[1.4400729 ],
       [2.4712381 ],
       [0.16822293],
       [1.7029064 ],
       [2.04747   ],
       [1.1755701 ],
       [1.5980489 ],
       [0.        ],
       [0.20402032],
       [1.4777297 ],
       [1.5900168 ],
       [2.1438358 ],
       [0.20186529],
       [1.4266883 ],
       [2.0835066 ],
       [0.        ],
       [1.0606157 ],
       [0.        ],
       [0.        ],
       [0.49948457],
       [0.58904487],
       [0.08472293],
       [0.36580038],
       [2.0389624 ],
       [2.2882586 ],
       [1.8173517 ],
       [1.0344427 ],
       [0.45675364],
       [0.9915461 ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.4686266 ],
       [0.27127826],
       [0.        ],
       [1.6224047 ],
       [0.32644144],
       [0.48913845],
       [0.24816355],
       [1.8140963 ],
       [0.        ],
       [0.        ],
       [0.85075223],
       [0.        ],
       [0.        ],
       [0.97794116],
       [1.889

In [23]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[1.],
       [2.],
       [0.],
       [2.],
       [2.],
       [1.],
       [2.],
       [0.],
       [0.],
       [1.],
       [2.],
       [2.],
       [0.],
       [1.],
       [2.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [2.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [2.],
       [3.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [2.],
       [2.],
       [0.],
       [0.],
       [0.],
       [1.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [2.],
       [1.],
       [0.],
       [2.],
       [0.],
       [2.],
       [1.],
       [2.],
       [2.],
       [2.],

In [24]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [25]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([156.], dtype=float32)

In [26]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

array([0.5051857], dtype=float32)

In [27]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

array([0.513577], dtype=float32)

In [28]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(test_x, test_y)
print(history.history.keys())
model.test_on_batch(test_x, test_y)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 3ms/step - loss: 1.2263 - mse: 1.2263 - mae: 0.8271 - root_mean_squared_error: 1.1074 - mean_squared_logarithmic_error: 0.4133
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [29]:
# Saving the model
model_id = 11
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model4/ant1.3_ant1.4_model4_3'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model4/ant1.3_ant1.4_model4_3/assets


In [30]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [31]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [32]:
score

[11,
 'ant-1.3',
 'ant-1.4',
 1.226317048072815,
 1.226317048072815,
 0.8270690441131592,
 1.1073919534683228,
 0.41329681873321533,
 0.5051857233047485,
 0.5135769844055176]

In [33]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [34]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)